In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai import ChatOpenAI

def add(a:int,b:int)->int:
    """ 
    Add a and b

    Args:
        a: first integer
        b: second integer
    """
    return a+b

def multiply(a:int,b:int)->int:
    """ 
    Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a*b 

def divide(a:int,b:int)->int:
    """ 
    Divide a and b.
    Args:
        a: first int
        b: second int
    """
    return a/b

tools = [add,multiply,divide]
llm = ChatOpenAI(model="gpt-4o")

llm_with_tools = llm.bind_tools(tools,parallel_tool_calls=False)

In [3]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages

class MessageState(TypedDict):
    messages: Annotated[list[AnyMessage],add_messages]

In [4]:
from langchain_core.messages import HumanMessage,SystemMessage

# system message
sys_msg = SystemMessage(content="You are a helpful assisstant tasked with performing arithmetic on a set of inputs")

def assisstant(state:MessageState):
    return {"messages":[llm_with_tools.invoke([sys_msg]+state["messages"])]}

In [5]:
from langgraph.graph import START,END,StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import display,Image

builder = StateGraph(MessageState)

builder.add_node("assisstant",assisstant)
builder.add_node("tools",ToolNode(tools))

builder.add_edge(START,"assisstant")
builder.add_conditional_edges(
    "assisstant",
    tools_condition
)

builder.add_edge("tools","assisstant")

react_graph = builder.compile()

#display(Image(react_graph.get_graph().draw_mermaid_png()))

In [6]:
messages = [HumanMessage(content="Add 10 and 14.multiply the output by 2.divide the output by 5")]

react_graph.invoke({"messages":messages})

Failed to get info from https://api.smith.langchain.com: LangSmithConnectionError('Connection error caused failure to GET /info in LangSmith API. Please confirm your internet connection. SSLError(MaxRetryError("HTTPSConnectionPool(host=\'api.smith.langchain.com\', port=443): Max retries exceeded with url: /info (Caused by SSLError(SSLCertVerificationError(1, \'[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1020)\')))"))\nContent-Length: None\nAPI Key: lsv2_********************************************09')


APIConnectionError: Connection error.

Failed to batch ingest runs: langsmith.utils.LangSmithConnectionError: Connection error caused failure to POST https://api.smith.langchain.com/runs/batch in LangSmith API. Please confirm your internet connection. SSLError(MaxRetryError("HTTPSConnectionPool(host='api.smith.langchain.com', port=443): Max retries exceeded with url: /runs/batch (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1020)')))"))
Content-Length: 39198
API Key: lsv2_********************************************09
post: trace=fafad3ca-6289-41c9-9725-ec5e24b9958a,id=fafad3ca-6289-41c9-9725-ec5e24b9958a; trace=fafad3ca-6289-41c9-9725-ec5e24b9958a,id=7e547c16-1c6c-4201-a14d-58a5f6a914a3; trace=fafad3ca-6289-41c9-9725-ec5e24b9958a,id=b882b457-6982-41ab-a946-0e27f5f198c5; trace=fafad3ca-6289-41c9-9725-ec5e24b9958a,id=c2046f4b-1455-417c-8ac9-e79844c6cf10; trace=fafad3ca-6289-41c9-9725-ec5e24b9958a,id=ba594bfb-99fc-4ae